In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from flapper_sim import FlapperSim

f = FlapperSim(robot_pose=np.array([0, 0, 0.8, 1.57]))
position = f.get_output_measurement()
y_1 = position[0]
y_2 = position[1]
y_3 = position[2]

In [ ]:
position_log = []
target_position_log = []

In [ ]:
# Discrete time state space model of robot
T = 0.1  # Sampling time

A_d = np.array([[1, 0.1, 0.005],
                [0, 1, 0.1],
                [0, 0, 1]])

B_d = np.array([[0.00016666667], [0.005], [0.1]])

C_d = np.array([[1, 0, 0]])

In [ ]:
# Controller
F = 10**3 * np.array([[-3.375, -0.3375, -0.0225]])
L = np.array([[-3], [-25], [-100]])

In [ ]:
# Target path
P0 = 0.8
P1 = 1.2
t0 = 0
t1 = 100

In [ ]:
# Target state function
def get_target_x(time):
    if time <= t0:
        return np.array([[P0], [0], [0]])
    elif time > t1:
        return np.array([[P1], [0], [0]])
    elif time <= 0.5 * (t0 + t1):
        pos = 2 * (P1 - P0) * ((time - t0) / (t1 - t0)) ** 2 + P0
        vel = 4 * (P1 - P0) * (time - t0) / (t1 - t0) ** 2
        acc = 8 * (P1 - P0) / (t1 - t0) ** 2
        return np.array([[pos], [vel], [acc]])
    elif time > 0.5 * (t0 + t1):
        pos = P1 - 2 * (P1 - P0) * ((time - t1) / (t1 - t0)) ** 2
        vel = -4 * (P1 - P0) * (time - t1) / (t1 - t0) ** 2
        acc = -8 * (P1 - P0) / (t1 - t0) ** 2
        return np.array([[pos], [vel], [acc]])

In [ ]:
# Initial state estimates
x_hat_1 = np.array([[0], [0], position[0]])
x_hat_2 = np.array([[0], [0], position[1]])
x_hat_3 = np.array([[0], [0], position[2]])

In [ ]:
t = 0

for i in range(2000):
    target_x = get_target_x(t)  # Example target positions

    # print("target_x_1.shape:", target_x_1.shape)

    u_1 = F @ (x_hat_1)
    u_2 = F @ (x_hat_1)
    u_3 = F @ (x_hat_1 - target_x[0])

    # print(f"t: {t}, u_1: {u_1}, u_2: {u_2}, u_3: {u_3}")
    # raise SystemExit

    # print("(A_d + L @ C_d).shape:", (A_d + L @ C_d).shape)
    # print("x_hat_1.shape:", x_hat_1.shape)

    x_hat_1 = (A_d + (L @ C_d)) @ x_hat_1#+ B_d * u_1 - L * y_1
    x_hat_2 = (A_d + (L @ C_d)) @ x_hat_2 #+ B_d * u_2 - L * y_2
    x_hat_3 = (A_d + (L @ C_d)) @ x_hat_3 #+ B_d * u_3 - L * y_3

    # print(f"x_hat_1: {x_hat_1}, x_hat_2: {x_hat_2}, x_hat_3: {x_hat_3}")
    # print(f"x_hat_1.shape: {x_hat_1.shape}, x_hat_2.shape: {x_hat_2.shape}, x_hat_3.shape: {x_hat_3.shape}")

    # print(f"u_1.shape: {u_1.shape}, u_2.shape: {u_2.shape}, u_3.shape: {u_3.shape}")

    f.step(u=np.array([u_1[0], u_2[0], u_3[0]]))

    position = f.get_output_measurement()

    position_log.append(position)
    target_position_log.append([[0], [0], target_x[0]])

    y_1 = position[0]
    y_2 = position[1]
    y_3 = position[2]

    t += T

In [ ]:
# position_log = np.array(position_log)
# target_position_log = np.array(target_position_log)
# print(target_position_log)

In [ ]:
# print(position_log.shape)

In [ ]:
import plotly.graph_objects as go

# Plot the log of output measurements in 3d
fig = go.Figure()
fig = go.Figure(data=[go.Scatter3d(
    x=np.array(position_log)[:, 0, 0],
    y=np.array(position_log)[:, 1, 0],
    z=np.array(position_log)[:, 2, 0],
    mode='lines+markers',
    marker=dict(size=4),
    line=dict(width=2)
)])
fig.add_trace(go.Scatter3d(
    x=np.array(target_position_log)[:, 0, 0],
    y=np.array(target_position_log)[:, 1, 0],
    z=np.array(target_position_log)[:, 2, 0],
    mode='lines+markers',
    marker=dict(size=4, color='red'),
    line=dict(width=2, color='red')
))
fig.update_layout(
    title='Output Measurements Over Time',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z',
        xaxis=dict(range=[-1.0, 2.0]),
        yaxis=dict(range=[-2.0, 2.0]),
        zaxis=dict(range=[0.4, 1.5])
    ),
    width=800,
    height=600
)
fig.show()